In [18]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pyproj
import shapely
from typing import List

In [19]:
# Set notebook display options
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [20]:
def geod2utm(row):
    '''   Convert geodetic coordinates to UTM   '''
    zn = '16T'
    datum = 'WGS84'
    lat = row['lat']
    lon = row['lon']
        
    p = pyproj.Proj(proj='utm', zone=zn, ellps=datum)
    
    X, Y = p(lon, lat)
    
    return pd.Series({'UTMx': X, 'UTMy': Y})

In [21]:
def make_utm_points(row):
    UTMx = row['UTMx']
    UTMy = row['UTMy']
    UTMPoint = shapely.geometry.Point(UTMx, UTMy)
    return UTMPoint

# Read in summary data from the 2010 census

In [22]:
census_summary_df = pd.read_csv('data/2010-census-summary.csv')

In [23]:
for i in range(67,129):
    del census_summary_df['Unnamed: {0}'.format(i)]

# Read in shapes of community areas

In [24]:
communities = gpd.read_file('data/community-areas.geojson')

In [25]:
census_summary_df.to_pickle('data/census_summary-cleaned.pkl')

In [26]:
communities_df = pd.read_json('data/community-areas.geojson')

In [27]:
community_names: List[str] = []
community_area_numbers: List[int] = []
community_geometries: List[shapely.geometry.Polygon] = []
community_modeled: pd.DataFrame = pd.DataFrame([{'community name': 'DELETE THIS',
                                                 'community number': 0,
                                                 'UTMGeometry': 'DELETE THIS'
                                                }])

In [28]:
del community_modeled
# community_modeled_dict: dict = {}
community_names: list = []
community_area_numbers: list = []
community_geometries:list = []
community_area_sizes: list = []
for feature in communities_df['features']:
    community_poly_points: list = []
    community_names.append(feature['properties']['community'])
    community_area_numbers.append(feature['properties']['area_numbe'])
    for point in feature['geometry']['coordinates'][0][0]:
        community_poly_points.append((geod2utm(pd.Series({'lon': point[0], 'lat': point[1]}))))
    community_polygon = shapely.geometry.Polygon(community_poly_points)
    community_geometries.append(community_polygon)
    community_area_sizes.append(community_polygon.area)

In [29]:
community_modeled = pd.DataFrame({'community name': community_names,
                                  'community number': community_area_numbers,
                                  'UTMGeometry': community_geometries,
                                  'community area': community_area_sizes
                                 },
                                 index=range(len(community_names)))

In [30]:
community_modeled['community number'] = community_modeled['community number'].astype('int64')

In [31]:
print(community_modeled.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 4 columns):
community name      77 non-null object
community number    77 non-null int64
UTMGeometry         77 non-null object
community area      77 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.5+ KB
None


In [32]:
community_modeled.to_pickle('data/community_modeled.pkl')

In [33]:
census_detail_df = pd.read_csv('data/il000012010.dp')
census_detail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Columns: 191 entries, DPST to 2.38
dtypes: float64(9), int64(180), object(2)
memory usage: 4.6+ MB


In [34]:
census_detail_df.head()

,DPST,IL,000,01,0000001,12830632,835577,859405,879448,922092,878964,910273,865684,855790,870100,940094,930785,807778,665429,485060,364475,289385,235381,234912,6292276,425893,438556,449160,473736,447513,456887,433782,426476,432756,463423,455953,392322,318903,225894,163405,122706,92019,72892,6538356,409684,420849,430288,448356,431451,453386,431902,429314,437344,476671,474832,415456,346526,259166,201070,166679,143362,162020,36.6,35.2,37.9,10072849,4884648,5188201,9701453,4693917,5007536,9154497,4412627,4741870,1990462,858966,1131496,1609213,676916,932297,12830632.1,12540650,9177877,1866414,43963,586934,188328,104166,114724,17542,61469,25036,75669,4050,1122,1100,492,1336,861412,289982,32749,56258,70615,66820,9423048,1974113,101451,668694,13546,958603,12830632.2,2027578,1602403,182989,22541,219645,10803054,12830632.3,2027578.1,1010124,33490,25114,6348,1073,845404,106025,10803054.1,8167753,1832924,18849,580586,2977,16008,183957,12830632.4,12528859,4836972,2332995,3840210,2753355,839861,318042,113738,678821,48169,24850,306163,301773,159989,99944,60045,141784,74977,66807,4836972.1,3182984,1455656,2332995.1,1016456,227684,104931,622305,334269,1653988,1346312,595401,130031,750911,338825,1619862,1168353,2.59,3.20,5296715,4836972.2,459743,158882,7998,82739,16677,47289,146158,2.5,9.1,4836972.3,3263639,1573333,8787463,3741396,2.69,2.38
0,DPST,IL,0,1,2,12222213,800946,822588,841555,882732,846174,875131,830935,819627,832056,894405,883930,764758,627456,453869,339023,269313,219334,218381,5983830,408049,419689,429773,453163,429381,437545,414974,407048,412542,439661,432019,370469,300008,211152,151255,113883,85542,67677,6238383,392897,402899,411782,429569,416793,437586,415961,412579,419514,454744,451911,394289,327448,242717,187768,155430,133792,150704,36.3,35.0,37.6,9581712,4636347,4945365,9226600,4453962,4772638,8701451,4184510,4516941,1859039,800601,1058438,1499920,629509,870411,12222213,11938039,8601683,1849543,42638,584992,187970,103813,114200,17428,61217,24924,75440,3909,1071,1068,476,1294,855274,284174,30628,55568,68914,66113,8841329,1955221,97748,665884,13221,951564,12222213,2013020,1592097,182169,22294,216460,10209193,12222213,2013020,1002934,33272,24879,6313,1053,839497,105072,10209193,7598749,1816271,17759,578679,2856,15777,179102,12222213,11946547,4592560,2202904,3681342,2634258,817346,307253,110994,652395,44803,23506,290763,275666,136153,80207,55946,139513,73530,65983,4592560,3018554,1391305,2202904,971667,216608,98641,599042,320997,1574006,1277022,564182,120920,712840,314992,1548253,1092079,2.60,3.22,5016899,4592560,424339,153500,7511,79094,15174,36921,132139,2.5,9.1,4592560,3075032,1517528,8329381,3617166,2.71,2.38
1,DPST,IL,0,1,3,11159069,739531,758714,776102,806339,765983,811181,770862,758960,767098,817507,804300,692322,565195,404313,299489,237529,192991,190653,5454750,376658,387075,396323,413836,387528,403869,383538,375562,379351,400726,392013,334243,269485,187461,133046,100015,74968,59053,5704319,362873,371639,379779,392503,378455,407312,387324,383398,387747,416781,412287,358079,295710,216852,166443,137514,118023,131600,36.0,34.7,37.3,8723064,4211799,4511265,8396240,4044127,4352113,7923696,3801671,4122025,1647867,708067,939800,1324975,554543,770432,11159069,10892300,7629003,1803592,40052,576802,186006,102140,112810,16962,60297,24396,74191,3681,987,997,460,1237,839170,266769,26904,53618,61652,63793,7852283,1900621,90436,655149,12484,932448,11159069,1968610,1557342,179255,21762,210251,9190459,11159069,1968610,978935,32440,24268,6205,1005,824129,101628,9190459,6650068,1771152,15784,570597,2676,15041,165141,11159069,10933902,4165560,1992864,3405183,2427026,777924,288881,106229,592371,38836,21378,261225,225167,103531,57874,45657,121636,64660,56976,4165560,2744357,1279097,1992864,897714,197419,87567,554074,293816,1421203,1151058,507450,106339,643608,276296,1423810,970458,2.62,3.25,4545115,4165560,379555,141806,6859,72356,13034,30259,115241,2.5,9.2,4165560,2770389,1395171,7581221,3352681,2.74,2.40
2,DPST,IL,0,1,4,4362551,298540,277200,2

In [52]:
census_tracts_df = gpd.read_file('data/CensusTracts-2010.geojson')
census_tracts_df['geoid10'] = census_tracts_df['geoid10'].astype('int64')
census_tracts_df.head()

,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10,geometry
0,17,8424,44,Census Tract 8424,44,17031842400,,842400,031,(POLYGON ((-87.62404799998049 41.7302169999839...
1,17,8403,59,Census Tract 8403,59,17031840300,,840300,031,(POLYGON ((-87.6860799999848 41.82295600001154...
2,17,8411,34,Census Tract 8411,34,17031841100,,841100,031,(POLYGON ((-87.62934700001182 41.8527970000265...
3,17,8412,31,Census Tract 8412,31,17031841200,,841200,031,(POLYGON ((-87.68813499997718 41.8556909999909...
4,17,8390,32,Census Tract 8390,32,17031839000,,839000,031,(POLYGON ((-87.63312200003458 41.8744880000269...


In [81]:
age_sex_race_relationship_housing_df = pd.read_csv('data/DEC_10_SF1_SF1DP1_with_ann.csv')

In [82]:
del age_sex_race_relationship_housing_df['Id']
del age_sex_race_relationship_housing_df['Number; RACE - Total population']
del age_sex_race_relationship_housing_df['Percent; RACE - Total population']
del age_sex_race_relationship_housing_df['Number; RELATIONSHIP - Total population']
del age_sex_race_relationship_housing_df['Percent; RELATIONSHIP - Total population']
del age_sex_race_relationship_housing_df['Number; HOUSEHOLDS BY TYPE - Total households']
del age_sex_race_relationship_housing_df['Percent; HOUSEHOLDS BY TYPE - Total households']
del age_sex_race_relationship_housing_df['Percent; HOUSEHOLDS BY TYPE - Total households - Average household size']
del age_sex_race_relationship_housing_df['Percent; HOUSEHOLDS BY TYPE - Total households - Average family size [7]']
del age_sex_race_relationship_housing_df['Percent; HOUSING OCCUPANCY - Total housing units - Homeowner vacancy rate (percent) [8]']
del age_sex_race_relationship_housing_df['Percent; HOUSING OCCUPANCY - Total housing units - Rental vacancy rate (percent) [9]']
del age_sex_race_relationship_housing_df['Percent; HOUSING TENURE - Occupied housing units - Owner-occupied housing units - Population in owner-occupied housing units']
del age_sex_race_relationship_housing_df['Percent; HOUSING TENURE - Occupied housing units - Owner-occupied housing units - Average household size of owner-occupied units']
del age_sex_race_relationship_housing_df['Percent; HOUSING TENURE - Occupied housing units - Renter-occupied housing units - Population in renter-occupied housing units']
del age_sex_race_relationship_housing_df['Percent; HOUSING TENURE - Occupied housing units - Renter-occupied housing units - Average household size of renter-occupied units']

In [83]:
age_sex_race_relationship_housing_df.rename(columns={'Id2': 'tract id',
                                                     'Geography': 'tract description',
                                                     'Number; SEX AND AGE - Total population': 'total population - number',
                                                     'Percent; SEX AND AGE - Total population': 'total population - percent'
                                                    }, inplace=True)
age_sex_race_relationship_housing_df.head()

,tract id,tract description,total population - number,total population - percent,Number; SEX AND AGE - Total population - Under 5 years,Percent; SEX AND AGE - Total population - Under 5 years,Number; SEX AND AGE - Total population - 5 to 9 years,Percent; SEX AND AGE - Total population - 5 to 9 years,Number; SEX AND AGE - Total population - 10 to 14 years,Percent; SEX AND AGE - Total population - 10 to 14 years,Number; SEX AND AGE - Total population - 15 to 19 years,Percent; SEX AND AGE - Total population - 15 to 19 years,Number; SEX AND AGE - Total population - 20 to 24 years,Percent; SEX AND AGE - Total population - 20 to 24 years,Number; SEX AND AGE - Total population - 25 to 29 years,Percent; SEX AND AGE - Total population - 25 to 29 years,Number; SEX AND AGE - Total population - 30 to 34 years,Percent; SEX AND AGE - Total population - 30 to 34 years,Number; SEX AND AGE - Total population - 35 to 39 years,Percent; SEX AND AGE - Total population - 35 to 39 years,Number; SEX AND AGE - Total population - 40 to 44 years,Percent; SEX AND AGE - Total population - 40 to 44 years,Number; SEX AND AGE - Total population - 45 to 49 years,Percent; SEX AND AGE - Total population - 45 to 49 years,Number; SEX AND AGE - Total population - 50 to 54 years,Percent; SEX AND AGE - Total population - 50 to 54 years,Number; SEX AND AGE - Total population - 55 to 59 years,Percent; SEX AND AGE - Total population - 55 to 59 years,Number; SEX AND AGE - Total population - 60 to 64 years,Percent; SEX AND AGE - Total population - 60 to 64 years,Number; SEX AND AGE - Total population - 65 to 69 years,Percent; SEX AND AGE - Total population - 65 to 69 years,Number; SEX AND AGE - Total population - 70 to 74 years,Percent; SEX AND AGE - Total population - 70 to 74 years,Number; SEX AND AGE - Total population - 75 to 79 years,Percent; SEX AND AGE - Total population - 75 to 79 years,Number; SEX AND AGE - Total population - 80 to 84 years,Percent; SEX AND AGE - Total population - 80 to 84 years,Number; SEX AND AGE - Total population - 85 years and over,Percent; SEX AND AGE - Total population - 85 years and over,Number; SEX AND AGE - Total population - Median age (years),Percent; SEX AND AGE - Total population - Median age (years),Number; SEX AND AGE - Total population - 16 years and over,Percent; SEX AND AGE - Total population - 16 years and over,Number; SEX AND AGE - Total population - 18 years and over,Percent; SEX AND AGE - Total population - 18 years and over,Number; SEX AND AGE - Total population - 21 years and over,Percent; SEX AND AGE - Total population - 21 years and over,Number; SEX AND AGE - Total population - 62 years and over,Percent; SEX AND AGE - Total population - 62 years and over,Number; SEX AND AGE - Total population - 65 years and over,Percent; SEX AND AGE - Total population - 65 years and over,Number; SEX AND AGE - Male population,Percent; SEX AND AGE - Male population,Number; SEX AND AGE - Male population - Under 5 years,Percent; SEX AND AGE - Male population - Under 5 years,Number; SEX AND AGE - Male population - 5 to 9 years,Percent; SEX AND AGE - Male population - 5 to 9 years,Number; SEX AND AGE - Male population - 10 to 14 years,Percent; SEX AND AGE - Male population - 10 to 14 years,Number; SEX AND AGE - Male population - 15 to 19 years,Percent; SEX AND AGE - Male population - 15 to 19 years,Number; SEX AND AGE - Male population - 20 to 24 years,Percent; SEX AND AGE - Male population - 20 to 24 years,Number; SEX AND AGE - Male population - 25 to 29 years,Percent; SEX AND AGE - Male population - 25 to 29 years,Number; SEX AND AGE - Male population - 30 to 34 years,Percent; SEX AND AGE - Male population - 30 to 34 years,Number; SEX AND AGE - Male population - 35 to 39 years,Percent; SEX AND AGE - Male population - 35 to 39 years,Number; SEX AND AGE - Male population - 40 to 44 years,Percent; SEX AND AGE - Male population - 40 to 44 years,Number; SEX AND AGE - Male population - 45 to 49 years,Percent; SEX AND AGE - Male population - 45 

In [84]:
age_sex_race_relationship_housing_df.to_pickle('data/census-2010-age-sex-race-relationship-housing-transformed.pkl')